##**Asssignment2-PART-B-Section1-Sentence Classification with BERT**

**NAME :** Ganesh Kumaran Masilamani

**STUDENT ID :** 200434339

In [1]:
#libraries imported
import keras
import numpy as np
from keras.layers import Lambda, GlobalAveragePooling1D, Dense, Embedding
from keras import backend as K
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.layers import LSTM, RNN, Dropout, Input, LeakyReLU, Bidirectional,Conv1D, GlobalMaxPooling1D
from keras.layers.core import Dense
from keras.models import Model
from keras.utils import plot_model

In [2]:
pip install transformers

     |████████████████████████████████| 2.1MB 4.9MB/s 
     |████████████████████████████████| 870kB 28.1MB/s 
     |████████████████████████████████| 3.3MB 43.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=0fd21c230387a44351d458632c7dbabf9c822dd8f3325148e399b1907e048f4f
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


##**Preprocessing and Tokenization**

In [3]:
from transformers import DistilBertTokenizer, RobertaTokenizer 
import tqdm
distil_bert = 'distilbert-base-uncased' # Pick any desired pre-trained model

# Defining DistilBERT tokonizer
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,
                                                max_length=128, pad_to_max_length=True)

def tokenize(sentences, tokenizer, pad_length=128, pad_to_max_length=True ):
    if type(sentences) == str:
        inputs = tokenizer.encode_plus(sentences, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length, 
                                             return_attention_mask=True, return_token_type_ids=True)
        return np.asarray(inputs['input_ids'], dtype='int32'), np.asarray(inputs['attention_mask'], dtype='int32'), np.asarray(inputs['token_type_ids'], dtype='int32')
    input_ids, input_masks, input_segments = [],[],[]
    for sentence in sentences:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length, 
                                             return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')


In [4]:
inputs = tokenizer.tokenize("The capital of France is [MASK].")
print(inputs,'\n')

inputs = tokenizer.tokenize("This is a pretrained model.")
print(inputs,'\n')

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer)
print(ids)
print(masks,"\n")

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer, pad_to_max_length=False)
print(ids)
print(masks)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['the', 'capital', 'of', 'france', 'is', '[MASK]', '.'] 

['this', 'is', 'a', 'pre', '##train', '##ed', 'model', '.'] 

[ 101 1996 3007 1997 2605 2003  103 1012  102    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


##**Loading Data**

###**Downloading and preprocessing data**

In [5]:
import requests
def downloadfile(url):
  rq = requests.get(url)
  open(url.split('/')[-1], 'wb').write(rq.content)

downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2016devtest-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2016dev-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2016test-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2016train-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2015test-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2015train-BD.tsv')

downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/gold/SemEval2017-task4-test.subtask-BD.english.txt')

with open('twitter-2016dev-BD.tsv', 'r') as f:
  dev_original = [l.strip().split('\t') for l in f.readlines()]
with open('SemEval2017-task4-test.subtask-BD.english.txt', 'r') as f:
  test_original = [l.strip().split('\t') for l in f.readlines()]
train_original = []
with open('twitter-2016train-BD.tsv', 'r') as f:
  train_original = [l.strip().split('\t') for l in f.readlines()]
with open('twitter-2016test-BD.tsv', 'r') as f:
  train_original.extend([l.strip().split('\t') for l in f.readlines()])
with open('twitter-2016devtest-BD.tsv', 'r') as f:
  train_original.extend([l.strip().split('\t') for l in f.readlines()])
with open('twitter-2015test-BD.tsv', 'r') as f:
  train_original.extend([l.strip().split('\t') for l in f.readlines() if l.strip().split('\t')[2] in ['negative','positive']])
with open('twitter-2015train-BD.tsv', 'r') as f:
  train_original.extend([l.strip().split('\t') for l in f.readlines() if l.strip().split('\t')[2] in ['negative','positive']])

print("Training entries: {}".format(len(train_original)))
print("Development entries: {}".format(len(dev_original)))
print("Testing entries: {}".format(len(test_original)))

Training entries: 17639
Development entries: 1325
Testing entries: 6185


In [6]:
print("ID \t TOPIC \t LABLE \t TWEET_TEXT")
print(train_original[0])
print(train_original[1])
print(train_original[2])
print(train_original[3])
print(train_original[4])

ID 	 TOPIC 	 LABLE 	 TWEET_TEXT
['628949369883000832', '@microsoft', 'negative', "dear @Microsoft the newOoffice for Mac is great and all, but no Lync update? C'mon."]
['628976607420645377', '@microsoft', 'negative', "@Microsoft how about you make a system that doesn't eat my friggin discs. This is the 2nd time this has happened and I am so sick of it!"]
['629023169169518592', '@microsoft', 'negative', "I may be ignorant on this issue but... should we celebrate @Microsoft's parental leave changes? Doesn't the gender divide suggest... (1/2)"]
['629179223232479232', '@microsoft', 'negative', 'Thanks to @microsoft, I just may be switching over to @apple.']
['629226490152914944', '@microsoft', 'positive', 'Microsoft, I may not prefer your gaming branch of business. But, you do make a damn fine operating system. #Windows10 @Microsoft']


In [7]:
# Please write your code to generate the following data
# your code goes here
#FOR TRAIN DATA
x_train_tweet_int= []
x_train_tweet_masks = []
x_train_topic_int = []
x_train_topic_masks = []
for train_data in range(len(train_original)):
  train_ids,train_masks,train_segments = tokenize(train_original[train_data][-1],tokenizer)
  #Append the train tweet_int and tweet_masks to ids and masks
  x_train_tweet_int.append(train_ids)
  x_train_tweet_masks.append(train_masks)
  train_ids,train_masks,train_segments = tokenize(train_original[train_data][1],tokenizer)
  #Append the train topic_int and topic_masks to ids and masks
  x_train_topic_int.append(train_ids)
  x_train_topic_masks.append(train_masks)

#FOR DEV DATA
x_dev_tweet_int= []
x_dev_tweet_masks = []
x_dev_topic_int = []
x_dev_topic_masks = []
for dev_data in range(len(dev_original)):
  dev_ids,dev_masks,dev_segments = tokenize(dev_original[dev_data][-1],tokenizer)
  #Append the dev tweet_int and tweet_masks to ids and masks
  x_dev_tweet_int.append(dev_ids)
  x_dev_tweet_masks.append(dev_masks)
  dev_ids,dev_masks,dev_segments = tokenize(dev_original[dev_data][1], tokenizer)
  #Append the dev topic_int and topic_masks to ids and masks
  x_dev_topic_int.append(dev_ids)
  x_dev_topic_masks.append(dev_masks)

#FOR TEST DATA
x_test_tweet_int= []
x_test_tweet_masks = []
x_test_topic_int = []
x_test_topic_masks = []
for test_data in range(len(test_original)):
  test_ids,test_masks,test_segments = tokenize(test_original[test_data][-1],tokenizer)
  #Append the test tweet_int and tweet_masks to ids and masks
  x_test_tweet_int.append(test_ids)
  x_test_tweet_masks.append(test_masks)
  test_ids,test_masks,test_segments = tokenize(test_original[test_data][1],tokenizer )
  #Append the test topic_int and topic_masks to ids and masks
  x_test_topic_int.append(test_ids)
  x_test_topic_masks.append(test_masks)


# If use the previous tokenize function, you can get a print result like:
assert len(x_train_topic_int) == len(train_original)
assert len(x_train_topic_masks) == len(x_train_topic_int)
assert len(x_test_topic_int) == len(test_original)
assert len(x_test_topic_masks) == len(x_test_topic_int)
print("x_dev_topic_int[0]:")
print(x_dev_topic_int[0])
print("x_dev_topic_masks[0]:")
print(x_dev_topic_masks[0])
print("x_dev_tweet_int[0]:")
print(x_dev_tweet_int[0])
print("x_dev_tweet_masks[0]:")
print(x_dev_tweet_masks[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


x_dev_topic_int[0]:
[ 101 2745 4027  102    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
x_dev_topic_masks[0]:
[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
x_dev_tweet_int[0]:
[  101  6108  1062  9794 1

We use 1 to represent "positive" and 0 for "negative" and generate the "y" data similar to previous labs.

In [8]:
def label2int(dataset):
  y = []
  for example in dataset:
    if example[2].lower() == "negative":
      y.append(0)
    else:
      y.append(1)
  return y
  
y_train = label2int(train_original)
y_dev = label2int(dev_original)
y_test = label2int(test_original)
y_train = np.array(y_train)
y_dev = np.array(y_dev)
y_test = np.array(y_test)

print(y_train[0])
print(y_train[1])
print(y_train[2])
print(y_train[3])
print(y_train[4])

0
0
0
0
1


We also generate these target labels using one hot encoding (This will be used in model 1).

In [9]:
def int2onehot(dataset):
  y = []
  for example in dataset:
    if example:
      y.append(np.array([0,1]))
    else:
      y.append(np.array([1,0]))
  return np.array(y)
y_train_onehot = int2onehot(y_train)
y_dev_onehot = int2onehot(y_dev)
y_test_onehot = int2onehot(y_test)

print(y_train_onehot[0])
print(y_train_onehot[1])
print(y_train_onehot[2])
print(y_train_onehot[3])
print(y_train_onehot[4])

[1 0]
[1 0]
[1 0]
[1 0]
[0 1]


Now we have almost done the data preprocessing. The easiest way is to input the tweet and topic as paired sentences into the model, concatenating them and separating them by the special BERT sentence-separator token [SEP] (available as tokenizer.sep_token). Then we can apply BERT directly.

In [10]:
# Please write your code to combine the tweet and topic into the following varibles
# your code goes here
#FOR TRAIN DATA (x_train_int, x_train_masks)
x_train_int= []
x_train_masks = []
for train_data_concatenate in range(len(train_original)):
  #Concatenating both train tweets and topics
  train_join = str(train_original[train_data_concatenate][1]) + tokenizer.sep_token + str(train_original[train_data_concatenate][-1])
  train_ids,train_masks,train_segments = tokenize(train_join,tokenizer)
  #Append the train_int and train_masks to ids and masks
  x_train_int.append(train_ids)
  x_train_masks.append(train_masks)

#FOR DEV DATA (x_dev_int, x_dev_masks)
x_dev_int= []
x_dev_masks = []
for dev_data_concatenate in range(len(dev_original)):
  #Concatenating both dev tweets and topics
  dev_join = str(dev_original[dev_data_concatenate][1]) + tokenizer.sep_token + str(dev_original[dev_data_concatenate][-1])
  dev_ids,dev_masks,dev_segments = tokenize(dev_join,tokenizer)
  #Append the dev_int and dev_masks to ids and masks
  x_dev_int.append(dev_ids)
  x_dev_masks.append(dev_masks)

#FOR TEST DATA (x_test_int, x_test_masks)
x_test_int= []
x_test_masks = []
for test_data_concatenate in range(len(test_original)):
  #Concatenating both test tweets and topics
  test_join = str(test_original[test_data_concatenate][1]) + tokenizer.sep_token + str(test_original[test_data_concatenate][-1])
  test_ids,test_masks,test_segments = tokenize(test_join,tokenizer)
  #Append the test_int and test_masks to ids and masks
  x_test_int.append(test_ids)
  x_test_masks.append(test_masks)

#Padding the train, dev, test integer and masked sequences
X_train_int = keras.preprocessing.sequence.pad_sequences(x_train_int, padding='post', maxlen=128)
X_dev_int = keras.preprocessing.sequence.pad_sequences(x_dev_int, padding='post', maxlen=128)
X_test_int = keras.preprocessing.sequence.pad_sequences(x_test_int, padding='post', maxlen=128)
X_train_masks = keras.preprocessing.sequence.pad_sequences(x_train_masks, padding='post', maxlen=128)
X_dev_masks = keras.preprocessing.sequence.pad_sequences(x_dev_masks, padding='post', maxlen=128)
X_train_masks = keras.preprocessing.sequence.pad_sequences(x_test_masks, padding='post', maxlen=128)

# Don't forget the to use np.array function to wrap the ouput of pad_sequences function
#Wrapping the output of the pad sequences
x_train_int_np = np.array(x_train_int)
x_train_masks_np = np.array(x_train_masks)
x_dev_int_np = np.array(x_dev_int)
x_dev_masks_np = np.array(x_dev_masks)
x_test_int_np = np.array(x_test_int)
x_test_masks_np = np.array(x_test_masks)


print(x_dev_int[0])
print(x_dev_masks[0],'\n')
print(x_dev_int_np[0])
print(x_dev_masks_np[0])


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[  101  2745  4027   102  6108  1062  9794 16021 23091  2007 16839  9080
 12863  7050  2000  2745  4027  1024  6108  1062  4593  2587 16021 23091
  2006  5095  1998  1012  1012  8299  1024  1013  1013  1056  1012  2522
  1013  1053  3501  2683  2072  2549  8586  2615 18037   102     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In your report, show a few examples of your input, with text and topic included but separated by [SEP].

In [14]:
#Examples of your inputs, with text and topic included but separated by [SEP]
#FOR TRAIN DATA
x_train= []
for train_data_concatenate in range(len(train_original)):
  #Concatenating both train tweets and topics using sep token
  train_join = str(train_original[train_data_concatenate][1]) + tokenizer.sep_token + str(train_original[train_data_concatenate][-1])
  train_inputs = tokenizer.tokenize(train_join)
  #Append the train data to its sentences
  x_train.append(train_inputs)

#FOR DEV DATA
x_dev= []
for dev_data_concatenate in range(len(dev_original)):
  #Concatenating both dev tweets and topics using sep token
  dev_join = str(dev_original[dev_data_concatenate][1]) + tokenizer.sep_token + str(dev_original[dev_data_concatenate][-1])
  dev_inputs = tokenizer.tokenize(dev_join)
  #Append the dev data to its sentences
  x_dev.append(dev_inputs)

#FOR TEST DATA
x_test= []
for test_data_concatenate in range(len(test_original)):
  #Concatenating both test tweets and topics using sep token
  test_join = str(test_original[test_data_concatenate][1]) + tokenizer.sep_token + str(test_original[test_data_concatenate][-1])
  test_inputs = tokenizer.tokenize(test_join)
  #Append the test data to its sentences
  x_test.append(test_inputs)

print(x_train[0])
print(x_dev[0])
print(x_test[1])

['@', 'microsoft', '[SEP]', 'dear', '@', 'microsoft', 'the', 'new', '##oof', '##fi', '##ce', 'for', 'mac', 'is', 'great', 'and', 'all', ',', 'but', 'no', 'l', '##yn', '##c', 'update', '?', 'c', "'", 'mon', '.']
['michael', 'jackson', '[SEP]', 'jay', 'z', 'joins', 'ins', '##tagram', 'with', 'nos', '##tal', '##gic', 'tribute', 'to', 'michael', 'jackson', ':', 'jay', 'z', 'apparently', 'joined', 'ins', '##tagram', 'on', 'saturday', 'and', '.', '.', 'http', ':', '/', '/', 't', '.', 'co', '/', 'q', '##j', '##9', '##i', '##4', '##ec', '##v', '##xy']
['#', 'aria', '##na', '##gra', '##nde', '[SEP]', 'aria', '##na', 'grande', 'white', 'house', 'easter', 'egg', 'roll', 'in', 'washington', 'https', ':', '/', '/', 't', '.', 'co', '/', 'jd', '##j', '##l', '##9', '##sw', '##w', '##m', '##8', '#', 'aria', '##na', '##gra', '##nde']


#**Model 1: Prebuilt Sequence Classification**



In [15]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig
import tensorflow as tf

distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(num_labels=2)
config.output_hidden_states = False

def create_TFDistilBertForSequenceClassification():
  transformer_model = TFDistilBertForSequenceClassification.from_pretrained(distil_bert, config = config)
  input_ids = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_ids = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')
  X = transformer_model(input_ids, input_masks_ids)
  return tf.keras.Model(inputs=[input_ids, input_masks_ids], outputs = X)

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)

  # Create model on TPU:
  with strategy.scope():
    model = create_TFDistilBertForSequenceClassification()
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model = create_TFDistilBertForSequenceClassification()
  model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.102.219.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.102.219.234:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_for_sequence_cla TFSequenceClassifier 66955010    input_token[0][0]                
                                                                 masked_token[0][0]               
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
__________________________________________________________________________________________________


In [ ]:
history = model.fit([x_train_int_np,x_train_masks_np],
                    y_train_onehot,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev_onehot),
                    verbose=1)

Epoch 1/30


35/35 [==============================] - ETA: 0s - loss: 0.9012 - accuracy: 0.5321WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


35/35 [==============================] - 90s 1s/step - loss: 0.8958 - accuracy: 0.5344 - val_loss: 0.5275 - val_accuracy: 0.7442
Epoch 2/30
35/35 [==============================] - 7s 206ms/step - loss: 0.4684 - accuracy: 0.7886 - val_loss: 0.3995 - val_accuracy: 0.7577
Epoch 3/30
35/35 [==============================] - 7s 207ms/step - loss: 0.3574 - accuracy: 0.8434 - val_loss: 0.3527 - val_accuracy: 0.8521
Epoch 4/30
35/35 [==============================] - 7s 206ms/step - loss: 0.2990 - accuracy: 0.8929 - val_loss: 0.3822 - val_accuracy: 0.8551
Epoch 5/30
35/35 [==============================] - 7s 206ms/step - loss: 0.2584 - accuracy: 0.9113 - val_loss: 0.3630 - val_accuracy: 0.8415
Epoch 6/30
35/35 [==============================] - 7s 206ms/step - loss: 0.2240 - accuracy: 0.9203 - val_loss: 0.4778 - val_accuracy: 0.8513
Epoch 7/30
35/35 [==============================] - 7s 206ms/step - loss: 0.1908 - accuracy: 0.9431 - val_loss: 0.4655 - val_accuracy: 0.8468
Epoch 8/30
35/35 [=

In [ ]:
results = model.evaluate([x_test_int_np,x_test_masks_np], y_test_onehot)
print(results)

194/194 [==============================] - 9s 31ms/step - loss: 1.3371 - accuracy: 0.8372
[1.3371375799179077, 0.8371867537498474]


##**INFERENCE:**
##**Comparison of Lab5 with model1 (Prebuilt Sequence Classification model)**

*Accuracy of the Lab5 (Neural bag of words without pre-trained word embeddings model)* -- **0.7528**

*Accuracy of the Lab5 (Neural bag of words with pre-trained word embeddings model)* -- **0.7876**

*Accuracy of the Lab5 (Neural bag of words model with multiple-input)* -- **0.7738**

*Accuracy with model1(Prebuilt Sequence Classification)* -- **0.8372**

##**Reason:**
* The model1 accuracy which is the Prebuilt Sequence Classification model acheives **high accuracy** than models which was created in lab5. This is because BERT is bidirectional which combines Masks with the sentence prediction. it is used in predicting the missing words in the sentence.



#**Model 2: Neural bag of words using BERT**

In [ ]:
class GlobalAveragePooling1DMasked(GlobalAveragePooling1D):
    def call(self, x, mask=None):
        if mask != None:
            return K.sum(x, axis=1) / K.sum(mask, axis=1)
        else:
            return super().call(x)

In [ ]:
from transformers import TFDistilBertModel, DistilBertConfig

def get_BERT_layer():
  distil_bert = 'distilbert-base-uncased'
  config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
  config.output_hidden_states = False
  return TFDistilBertModel.from_pretrained(distil_bert, config = config)


In [ ]:
hdepth=16
MAX_SEQUENCE_LENGTH = 128
EMBED_SIZE=100


def create_bag_of_words_BERT():
  input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

  bert_embeddings = get_BERT_layer()
  embedded_sent = bert_embeddings(input_ids_in, attention_mask=input_masks_in)[0]

  pooled_sent=GlobalAveragePooling1DMasked()(embedded_sent)
  hidden_output=Dense(hdepth,input_shape=(MAX_SEQUENCE_LENGTH,EMBED_SIZE),activation='sigmoid',kernel_initializer='glorot_uniform')(pooled_sent) # Sigmoid
  label=Dense(1,input_shape=(hdepth,),activation='sigmoid',kernel_initializer='glorot_uniform')(hidden_output)
  return Model(inputs=[input_ids_in,input_masks_in], outputs=[label],name='Model2_BERT')

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)

  # Create model
  with strategy.scope():
    model2 = create_bag_of_words_BERT()
    optimizer2 = keras.optimizers.Adam(lr=5e-5)
    model2.compile(optimizer=optimizer2, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model2 = create_bag_of_words_BERT()
  model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model2.summary() 

INFO:tensorflow:Initializing the TPU system: grpc://10.37.91.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.37.91.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertMo

Model: "Model2_BERT"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB TFBaseModelOutput(la 66362880    input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
global_average_pooling1d_masked (None, 768)          0           tf_distil_bert_model[0]

In [ ]:
history = model2.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30


35/35 [==============================] - ETA: 0s - loss: 0.4339 - accuracy: 0.8169WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


35/35 [==============================] - 92s 1s/step - loss: 0.4319 - accuracy: 0.8177 - val_loss: 0.3266 - val_accuracy: 0.8619
Epoch 2/30
35/35 [==============================] - 7s 204ms/step - loss: 0.2545 - accuracy: 0.9027 - val_loss: 0.3213 - val_accuracy: 0.8528
Epoch 3/30
35/35 [==============================] - 7s 207ms/step - loss: 0.2012 - accuracy: 0.9329 - val_loss: 0.3229 - val_accuracy: 0.8513
Epoch 4/30
35/35 [==============================] - 7s 205ms/step - loss: 0.1667 - accuracy: 0.9524 - val_loss: 0.3535 - val_accuracy: 0.8430
Epoch 5/30
35/35 [==============================] - 7s 204ms/step - loss: 0.1437 - accuracy: 0.9646 - val_loss: 0.3400 - val_accuracy: 0.8558
Epoch 6/30
35/35 [==============================] - 7s 203ms/step - loss: 0.1286 - accuracy: 0.9722 - val_loss: 0.3692 - val_accuracy: 0.8438
Epoch 7/30
35/35 [==============================] - 7s 207ms/step - loss: 0.1188 - accuracy: 0.9764 - val_loss: 0.3441 - val_accuracy: 0.8528
Epoch 8/30
35/35 [=

In [ ]:
results = model2.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

194/194 [==============================] - 8s 31ms/step - loss: 0.4170 - accuracy: 0.8863
[0.41697075963020325, 0.8863378763198853]


##**INFERENCE:**
##**Comparison of model1 and Lab5 with model2 (Neural bag of words using BERT model)**

*Accuracy with model1(Prebuilt Sequence Classification)* -- **0.8372**

*Accuracy of the Lab5 (Neural bag of words without pre-trained word embeddings model)* -- **0.7528**

*Accuracy of the Lab5 (Neural bag of words with pre-trained word embeddings model)* -- **0.7876**

*Accuracy of the Lab5 (Neural bag of words model with multiple-input)* -- **0.7738**

*Accuracy with model2 (Neural bag of words using BERT)*  -- **0.8863**

##**Reason:**
* The accuracy of model2 which is the Neural bag of words using BERT model acheives **high accuracy** than Prebuilt Sequence Classification model and the model which was developed in lab5. This model produces high accuracy because it is bidirectional which combines Masks with the sentence prediction. it is used in predicting the missing words in the sentence.


#**Model 3: CNN or LSTM with BERT**

#**LSTM model with BERT**

In [ ]:
# your code goes here
hdepth=16
MAX_SEQUENCE_LENGTH = 128
EMBED_SIZE=100

def create_bag_of_words_BERT_LSTM():
  input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

  bert_embeddings = get_BERT_layer()
  #Embedding layer
  embedded_sent = bert_embeddings(input_ids_in, attention_mask=input_masks_in)[0]
  #LSTM layer
  lstm_layer  = LSTM(150, return_sequences=False)(embedded_sent)
  #Hidden layer
  hidden_output=Dense(hdepth,input_shape=(MAX_SEQUENCE_LENGTH,EMBED_SIZE),activation='sigmoid',kernel_initializer='glorot_uniform')(lstm_layer)
  #Output layer 
  label=Dense(1,input_shape=(hdepth,),activation='sigmoid',kernel_initializer='glorot_uniform')(hidden_output)
  return Model(inputs=[input_ids_in,input_masks_in], outputs=[label],name='Model2_BERT')

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)

  # Create model
  with strategy.scope():
    model3a = create_bag_of_words_BERT_LSTM()
    optimizer2 = keras.optimizers.Adam(lr=5e-5)
    model3a.compile(optimizer=optimizer2, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model3a = create_bag_of_words_BERT_LSTM()
  model3a.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model3a.summary()

INFO:tensorflow:Initializing the TPU system: grpc://10.37.91.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.37.91.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertMo

Model: "Model2_BERT"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_1 (TFDisti TFBaseModelOutput(la 66362880    input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 150)          551400      tf_distil_bert_model_1[

In [ ]:
#training the model
history = model3a.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30


35/35 [==============================] - ETA: 0s - loss: 0.6451 - accuracy: 0.6251WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


35/35 [==============================] - 95s 1s/step - loss: 0.6432 - accuracy: 0.6283 - val_loss: 0.5715 - val_accuracy: 0.7442
Epoch 2/30
35/35 [==============================] - 8s 216ms/step - loss: 0.5228 - accuracy: 0.7969 - val_loss: 0.5244 - val_accuracy: 0.7442
Epoch 3/30
35/35 [==============================] - 8s 215ms/step - loss: 0.4747 - accuracy: 0.8382 - val_loss: 0.4961 - val_accuracy: 0.7985
Epoch 4/30
35/35 [==============================] - 8s 217ms/step - loss: 0.4090 - accuracy: 0.8905 - val_loss: 0.4337 - val_accuracy: 0.8551
Epoch 5/30
35/35 [==============================] - 8s 218ms/step - loss: 0.3574 - accuracy: 0.9303 - val_loss: 0.4373 - val_accuracy: 0.8626
Epoch 6/30
35/35 [==============================] - 8s 219ms/step - loss: 0.3283 - accuracy: 0.9486 - val_loss: 0.4262 - val_accuracy: 0.8642
Epoch 7/30
35/35 [==============================] - 8s 217ms/step - loss: 0.3151 - accuracy: 0.9550 - val_loss: 0.4297 - val_accuracy: 0.8657
Epoch 8/30
35/35 [=

In [ ]:
#Evaluating on test data
results = model3a.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

194/194 [==============================] - 9s 34ms/step - loss: 0.3340 - accuracy: 0.8888
[0.3340151906013489, 0.8887631297111511]


#**CNN model with BERT**

In [ ]:
hdepth=16
MAX_SEQUENCE_LENGTH = 128
EMBED_SIZE=100

def create_bag_of_words_BERT_CNN():
  input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

  bert_embeddings = get_BERT_layer()
  #Embedding layer
  embedded_sent = bert_embeddings(input_ids_in, attention_mask=input_masks_in)[0]
  #CNN layer
  cnn_layer  = Conv1D(512, kernel_size = 8, activation="relu", strides=3)(embedded_sent)
  #Hidden layer
  hidden_output=Dense(hdepth,input_shape=(MAX_SEQUENCE_LENGTH,EMBED_SIZE),activation='sigmoid',kernel_initializer='glorot_uniform')(cnn_layer)
  #Output layer 
  label=Dense(1,input_shape=(hdepth,),activation='sigmoid',kernel_initializer='glorot_uniform')(hidden_output)
  return Model(inputs=[input_ids_in,input_masks_in], outputs=[label],name='Model2_BERT')

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)

  # Create model
  with strategy.scope():
    model3b = create_bag_of_words_BERT_CNN()
    optimizer2 = keras.optimizers.Adam(lr=5e-5)
    model3b.compile(optimizer=optimizer2, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model3b = create_bag_of_words_BERT_CNN()
  model3b.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model3b.summary()

INFO:tensorflow:Initializing the TPU system: grpc://10.37.91.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.37.91.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertMo

Model: "Model2_BERT"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_2 (TFDisti TFBaseModelOutput(la 66362880    input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 41, 512)      3146240     tf_distil_bert_model_2[

In [ ]:
#training the model
history = model3b.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30


35/35 [==============================] - ETA: 0s - loss: 0.4302 - accuracy: 0.8170WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


35/35 [==============================] - 97s 1s/step - loss: 0.4279 - accuracy: 0.8179 - val_loss: 0.3177 - val_accuracy: 0.8658
Epoch 2/30
35/35 [==============================] - 7s 210ms/step - loss: 0.2262 - accuracy: 0.9094 - val_loss: 0.3389 - val_accuracy: 0.8344
Epoch 3/30
35/35 [==============================] - 7s 211ms/step - loss: 0.1804 - accuracy: 0.9328 - val_loss: 0.3175 - val_accuracy: 0.8562
Epoch 4/30
35/35 [==============================] - 7s 210ms/step - loss: 0.1401 - accuracy: 0.9535 - val_loss: 0.3623 - val_accuracy: 0.8459
Epoch 5/30
35/35 [==============================] - 7s 209ms/step - loss: 0.1230 - accuracy: 0.9614 - val_loss: 0.4074 - val_accuracy: 0.8349
Epoch 6/30
35/35 [==============================] - 7s 211ms/step - loss: 0.0944 - accuracy: 0.9769 - val_loss: 0.4143 - val_accuracy: 0.8366
Epoch 7/30
35/35 [==============================] - 7s 210ms/step - loss: 0.0760 - accuracy: 0.9830 - val_loss: 0.4129 - val_accuracy: 0.8404
Epoch 8/30
35/35 [=

In [ ]:
#Evaluating on test data
results = model3b.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

194/194 [==============================] - 9s 33ms/step - loss: 0.3719 - accuracy: 0.8918
[0.3718743920326233, 0.8917599320411682]


#**INFERENCE:**
##**Comparison of model1 and model2 with model3(LSTM model):**

*Accuracy with model1(Prebuilt Sequence Classification)* -- **0.8372**

*Accuracy with model2 (Neural bag of words using BERT)*  -- **0.8863**

*Accuracy with model3 (LSTM model with BERT)*           -- **0.8888**
##**Reason:**
* The accuracy of model3 which is the LSTM model with BERT acheives high accuracy than Prebuilt Sequence Classification model and the Neural bags of words model.
* LSTM model acheives high accuracy because it is assigned with the LSTM units so that the backpropagation is performed better. although there is very minimal increase in the accuracy between model2 and model3. 

##**Comparison of model1 and model2 with model3(CNN model):**

*Accuracy with model1(Prebuilt Sequence Classification)* -- **0.8372**

*Accuracy with model2 (Neural bag of words using BERT)*  -- **0.8863**

*Accuracy with model3 (LSTM model with BERT)*           -- **0.8918**
##**Reason:**
* The model3 which is the CNN model with BERT acheives high accuracy than Prebuilt Sequence Classification model(model1) and the Neural bags of words model(model2).
* CNN model acheives high accuracy because the data is parsed through the convolutional layers and so that the backpropagation is performed better and the loss will be decreased gradually. 
